In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_ce07b42d61ad4121935133064ef4624e.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45.0,3


In [2]:
df.shape

(366, 35)

In [3]:
df.isna().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33     True
34    False
dtype: bool

In [4]:
df=df.fillna(df.median())
df.isna().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
dtype: bool

In [5]:
y=pd.DataFrame(df.iloc[:,34]).rename(columns={34:"Class"})
y.head()

,Class
0,2
1,1
2,3
3,1
4,3


In [6]:
y.groupby("Class")["Class"].count()

Class
1    112
2     61
3     72
4     49
5     52
6     20
Name: Class, dtype: int64

In [7]:
age=pd.DataFrame(df.iloc[:,33]).rename(columns={33:"age"})
age.head()

,age
0,55.0
1,8.0
2,26.0
3,40.0
4,45.0


In [8]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='uniform')
age=pd.DataFrame(est.fit_transform(age),columns=age.columns)
age.head()

,age
0,4.0
1,0.0
2,2.0
3,3.0
4,3.0


In [9]:
sub_X=df.drop(columns=[33,34])
sub_X.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2,2,0,3,0,0,0,0,1,0,...,0,0,0,0,3,0,0,0,1,0
1,3,3,3,2,1,0,0,0,1,1,...,1,0,0,0,0,0,0,0,1,0
2,2,1,2,3,1,3,0,3,0,0,...,0,2,0,2,3,2,0,0,2,3
3,2,2,2,0,0,0,0,0,3,2,...,0,0,3,0,0,0,0,0,3,0
4,2,3,2,2,2,2,0,2,0,0,...,0,2,2,3,2,3,0,0,2,3


In [10]:
X=pd.concat([sub_X,age],axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,age
0,2,2,0,3,0,0,0,0,1,0,...,0,0,0,3,0,0,0,1,0,4.0
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0.0
2,2,1,2,3,1,3,0,3,0,0,...,2,0,2,3,2,0,0,2,3,2.0
3,2,2,2,0,0,0,0,0,3,2,...,0,3,0,0,0,0,0,3,0,3.0
4,2,3,2,2,2,2,0,2,0,0,...,2,2,3,2,3,0,0,2,3,3.0


In [11]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
32,1201.968498
26,840.169359
28,730.394073
30,719.189172
5,627.241460
11,553.689327
14,483.219992
24,455.624896
7,368.617816
19,366.195320


In [12]:
X_scores=scores[scores["scores"]>=500]
len(X_scores)

6

In [13]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,32,26,28,30,5,11
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,3,2,2,0,3,1
3,0,0,0,0,0,0
4,3,3,3,0,2,1


In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [19]:
BForest=BalancedRandomForestClassifier(n_estimators=200,bootstrap=False,sampling_strategy="not majority",replacement=False,
                                       n_jobs=-1,warm_start=False,class_weight="balanced",random_state=0)
scores=cross_val_score(BForest,X,np.asarray(y).reshape(y.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9782282282282282 +/- 0.026513302462343527


In [20]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(BForest, X,np.asarray(y).reshape(y.shape[0],) , cv=10)

In [21]:
print("F1 score: ",f1_score(y,y_pred,average="micro"))

F1 score:  0.9781420765027322


In [22]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y,y_pred))

Confusion matrix: 
 [[112   0   0   0   0   0]
 [  1  56   0   3   0   1]
 [  0   0  72   0   0   0]
 [  0   3   0  46   0   0]
 [  0   0   0   0  52   0]
 [  0   0   0   0   0  20]]


In [23]:
from sklearn.metrics import classification_report
print("Classification report: \n",classification_report(y,y_pred))

Classification report: 
               precision    recall  f1-score   support

           1       0.99      1.00      1.00       112
           2       0.95      0.92      0.93        61
           3       1.00      1.00      1.00        72
           4       0.94      0.94      0.94        49
           5       1.00      1.00      1.00        52
           6       0.95      1.00      0.98        20

    accuracy                           0.98       366
   macro avg       0.97      0.98      0.97       366
weighted avg       0.98      0.98      0.98       366

